In [7]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
#from torchsummary import summary
import seaborn as sns
import sys
import torch.nn.functional as F
import pywt
from sklearn.preprocessing import MinMaxScaler
from torch import FloatTensor
from scipy.stats import skew, kurtosis, entropy
from math import gcd
from functools import reduce
import pywt
from scipy.stats import skew, kurtosis
from scipy.special import entr

# Req for package
sys.path.append("../")
from SkinLearning.NN.Helpers import train, test, set_seed, DEVICE
from SkinLearning.Utils.Dataset import get_dataset, get_split
from SkinLearning.NN.Models import MultiTemporal
import gc

In [8]:
set_seed()

In [9]:
dataset, scaler = get_dataset()

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:10<00:00, 217.30it/s]


In [10]:
train_loader, test_loader = get_split(dataset)

In [11]:
def get_models():
    return [

        MultiTemporal(
            out="f_output",
            hidden_size=64,
            temporal_type="LSTM"
        ),
        MultiTemporal(
            out="f_output",
            hidden_size=128,
            temporal_type="LSTM"
        ),
        MultiTemporal(
            out="f_output",
            hidden_size=256,
            temporal_type="LSTM"
        )
    ]

In [12]:
names = [64, 128, 256]

In [13]:
def get_avg(models, run=1):
    for i, model in enumerate(models):
        print(f"Training with {names[i]} hidden size for run {i}")
        train(train_loader, model, epochs=1500, early_stopping=True, val_loader=test_loader)
    
    maes = []
    for model in models:
        _, _, model_mae = test(test_loader, model, scaler)
        maes.append(model_mae)
        
    return maes
        
        

In [14]:
def repeat_avg(repeats=5):
    maes = []
    for i in range(repeats):
        models = get_models()
        
        single_maes = get_avg(models, run=i)
        maes.append(single_maes)
        
        del models
        torch.cuda.empty_cache()
        gc.collect()
    mean = np.mean(np.array(maes), axis=0)
    print(mean)
    
    return mean

In [15]:
np.mean(np.array([[1, 2, 3], [2, 2, 2]]), axis=0)

array([1.5, 2. , 2.5])

In [16]:
repeat_avg()

FC in: 64 HS 64
FC in: 128 HS 128
FC in: 256 HS 256
Training with 64 hidden size for run 0
Using: cuda


100%|██████████████████| 113/113 [00:01<00:00, 112.95batch/s, counter=43, epoch=68/1500, lastLoss=0.277, valLoss=0.518]


100%|██████████████████| 113/113 [00:00<00:00, 119.14batch/s, counter=49, epoch=74/1500, lastLoss=0.273, valLoss=0.309]


Early stopping after 74 epochs
Average train loss: 0.013605889002438143
Average validation loss: 0.0202035623655237
Training with 128 hidden size for run 1
Using: cuda


100%|███████████████████| 113/113 [00:01<00:00, 83.89batch/s, counter=48, epoch=68/1500, lastLoss=0.373, valLoss=0.447]


...

100%|████████████████████| 113/113 [00:02<00:00, 42.81batch/s, counter=28, epoch=68/1500, lastLoss=0.966, valLoss=1.15]


100%|██████████████████| 113/113 [00:02<00:00, 44.16batch/s, counter=22, epoch=136/1500, lastLoss=0.854, valLoss=0.738]


100%|███████████████████| 113/113 [00:02<00:00, 44.23batch/s, counter=49, epoch=163/1500, lastLoss=0.796, valLoss=1.55]


Early stopping after 163 epochs
Average train loss: 0.10241322970847466
Average validation loss: 0.11387982892305001
[       nan        nan 1.06976045]


array([       nan,        nan, 1.06976045])

Loss increases as hidden_size increases, 32 best

In [1]:
def get_models():
    return [
        MultiTemporal(
            out="f_output",
            hidden_size=8,
        ),
        MultiTemporal(
            out="f_output",
            hidden_size=16,
        ),
    ]

In [2]:
names = [8, 16]

In [3]:
repeat_avg()

NameError: name 'repeat_avg' is not defined

In [ ]:
def get_models():
    return [
        MultiTemporal(
            out="f_output",
            hidden_size=2,
        ),
        MultiTemporal(
            out="f_output",
            hidden_size=4,
        ),
    ]

In [ ]:
names = [2, 4]

In [ ]:
repeat_avg()